In [1]:
# %pip install llama-index
# %pip install llama-index-vector-stores-qdrant
# %pip install chromadb
# %pip install llama-index-vector-stores-chroma
# %pip install llama-index-embeddings-huggingface

In [2]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.core.readers import StringIterableReader
from llama_index.core.node_parser.text import SentenceWindowNodeParser

query = "在太享E保百万医疗险合同中，保险条款的作用是什么？"
raw_context = "产品名：太享E保百万医疗险\n条款：合同构成 本合同是您与我们约定保险权利义务关系的协议，包括本保险条款、保险单及其他保险凭证、投保单及其他投保文件、合法有效的声明、批注、批单及其他您与我们共同认可的书面协议。"

In [3]:
window_size = 3

documents = StringIterableReader().load_data(
    texts=[
        "产品名：太享E保百万医疗险\n条款：合同构成 本合同是您与我们约定保险权利义务关系的协议，包括本保险条款、保险单及其他保险凭证、投保单及其他投保文件、合法有效的声明、批注、批单及其他您与我们共同认可的书面协议。"
    ]
)
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=window_size,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents(documents)

len(nodes)

1

In [4]:
import chromadb

# import chromadb.utils.embedding_functions as embedding_functions
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./chroma_db")
# huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
#     model_name="BAAI/bge-m3", api_key=os.getenv("HUGGINGFACE_API_KEY")
# )

# create collection
chroma_collection = db.get_or_create_collection(
    f"bgem3_sentence_test_{window_size}",
    metadata={"hnsw:space": "cosine"},
    # embedding_function=huggingface_ef,
)

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model="local:BAAI/bge-m3",
    show_progress=True,
)

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
retriever = vector_index.as_retriever(similarity_top_k=1)
retrieved_nodes = retriever.retrieve(query)

In [11]:
retrieved_nodes[0].node.text

'产品名：太享E保百万医疗险\n条款：合同构成 本合同是您与我们约定保险权利义务关系的协议，包括本保险条款、保险单及其他保险凭证、投保单及其他投保文件、合法有效的声明、批注、批单及其他您与我们共同认可的书面协议。'